Продолжим работу с данными, которые были использованы в ДЗ2 и 3, продолжим решать задачу обнаружения мошеннических транзакций, что позволит получить полное решение задачи / полный пайплайн.

Задание 0: выбрать любую модель машнного обучения и зафиксировать любой тип валидации. Обучить базовую модель и зафиксировать базовое качество модели. В каждом следующем задании нужно будет обучить выбранную модель и оценивать ее качество на зафиксированной схеме валидации. После каждого задания, требуется сделать вывод о достигаемом качестве модели, по сравнению с качестом из предыдущего шага.

Задание 1: признак TransactionDT - это смещение в секундах относительно базовой даты. Базовая дата - 2017-12-01, преобразовать признак TransactionDT в datetime, прибавив к базовой дате исходное значение признака. Из полученного признака выделить год, месяц, день недели, час, день.

Задание 2: сделать конкатенацию признаков
* card1 + card2;
* card1 + card2 + card_3 + card_5;
* card1 + card2 + card_3 + card_5 + addr1 + addr2

Рассматривать их как категориальных признаки.

Задание 3: Сделать FrequencyEncoder для признаков card1 - card6, addr1, addr2.

Задание 4: Создать признаки на основе отношения: TransactionAmt к вычисленной статистике. Статистика - среднее значение / стандартное отклонение TransactionAmt, сгруппированное по card1 - card6, addr1, addr2, и по признакам, созданным в задании 2.

Задание 5: Создать признаки на основе отношения: D15 к вычисленной статистике. Статистика - среднее значение / стандартное отклонение D15, сгруппированное по card1 - card6, addr1, addr2, и по признакам, созданным в задании 2.

Задание 6: выделить дробную часть и целую часть признака TransactionAmt в два отдельных признака. После создать отдельных признак - логарифм от TransactionAmt

Задание 7 (опция): выполнить предварительную подготовку / очистку признаков P_emaildomain и R_emaildomain (что и как делать - остается на ваше усмотрение) и сделать Frequency Encoding для очищенных признаков.

In [276]:
import warnings
from tqdm import tqdm
from typing import List, Tuple

import numpy as np
import pandas as pd
import seaborn as sns
import xgboost as xgb
import matplotlib.pyplot as plt
from scipy.stats import ttest_rel, probplot
from sklearn.model_selection import TimeSeriesSplit
import catboost as cb

from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, cross_val_score, ShuffleSplit, learning_curve
import missingno as msno
warnings.simplefilter("ignore")
%matplotlib inline

import gc # сборщик мусора
import pickle

In [277]:
def create_bootstrap_samples(data: np.array, n_samples: int = 1000) -> np.array:
    """
    Создание бутстреп-выборок.

    Parameters
    ----------
    data: np.array
        Исходная выборка, которая будет использоваться для
        создания бутстреп выборок.

    n_samples: int, optional, default = 1000
        Количество создаваемых бутстреп выборок.
        Опциональный параметр, по умолчанию, равен 1000.

    Returns
    -------
    bootstrap_idx: np.array
        Матрица индексов, для создания бутстреп выборок.

    """
    bootstrap_idx = np.random.randint(
        low=0, high=len(data), size=(n_samples, len(data))
    )
    return bootstrap_idx


def create_bootstrap_metrics(y_true: np.array,
                             y_pred: np.array,
                             metric: callable,
                             n_samlpes: int = 1000) -> List[float]:
    """
    Вычисление бутстреп оценок.

    Parameters
    ----------
    y_true: np.array
        Вектор целевой переменной.

    y_pred: np.array
        Вектор прогнозов.

    metric: callable
        Функция для вычисления метрики.
        Функция должна принимать 2 аргумента: y_true, y_pred.

    n_samples: int, optional, default = 1000
        Количество создаваемых бутстреп выборок.
        Опциональный параметр, по умолчанию, равен 1000.

    Returns
    -------
    bootstrap_metrics: List[float]
        Список со значениями метрики качества на каждой бустреп выборке.

    """
    scores = []

    if isinstance(y_true, pd.Series):
        y_true = y_true.values

    bootstrap_idx = create_bootstrap_samples(y_true)
    for idx in bootstrap_idx:
        y_true_bootstrap = y_true[idx]
        y_pred_bootstrap = y_pred[idx]

        score = metric(y_true_bootstrap, y_pred_bootstrap)
        scores.append(score)

    return scores


def calculate_confidence_interval(scores: list, conf_interval: float = 0.95) -> Tuple[float]:
    """
    Вычисление доверительного интервала.

    Parameters
    ----------
    scores: List[float / int]
        Список с оценками изучаемой величины.

    conf_interval: float, optional, default = 0.95
        Уровень доверия для построения интервала.
        Опциональный параметр, по умолчанию, равен 0.95.

    Returns
    -------
    conf_interval: Tuple[float]
        Кортеж с границами доверительного интервала.

    """
    left_bound = np.percentile(
        scores, ((1 - conf_interval) / 2) * 100
    )
    right_bound = np.percentile(
        scores, (conf_interval + ((1 - conf_interval) / 2)) * 100
    )

    return left_bound, right_bound

In [278]:
PATH = 'C:/Users/Kokorozaci/Downloads/assignment2_data/'
train = pd.read_csv(PATH + "assignment_2_train.csv")
test = pd.read_csv(PATH + "assignment_2_test.csv")
print("data.shape = {} rows, {} cols".format(*train.shape))
train.head(n=2)

data.shape = 180000 rows, 394 cols


,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [279]:
y_train = train["isFraud"]
y_test = test["isFraud"]
x_train = train.set_index('TransactionID')
x_train = train.drop("isFraud", axis=1)
x_test = test.set_index('TransactionID')
x_test = test.drop("isFraud", axis=1)
print("data.shape = {} rows, {} cols".format(*train.shape), "data.shape = {} rows, {} cols".format(*test.shape))

data.shape = 180000 rows, 394 cols data.shape = 100001 rows, 394 cols


In [280]:
class FeatureImputer:
    """Обработка категориальных фич"""
    
    def __init__(self):
        self.numerical_features_col=None
        self.categorical_features_col=None
        self.bin_features_col = None
        self.m = None
        self.m_P_emaildomain = None
        self.m_ProductCD = None
        self.m_card4 = None
        self.m_card6 = None
        
    def fit(self, X):
        self.numerical_features_col = X.select_dtypes(include=[np.number]).columns
        self.categorical_features_col = train.select_dtypes('object').columns
        self.m = {f: {} for f in self.categorical_features_col}
        self.bin_features_col = []
        for feature in self.categorical_features_col:
            if len(X[feature].value_counts()) == 2:
                self.bin_features_col.append(feature)
                continue
            self.m[feature] = {v:k for k, v in enumerate(X.groupby(feature).count().index)}
        self.m['bin'] = {'F': 0, 'T': 1}
    
    def transform(self, X): 
        for feature in self.categorical_features_col:
            if len(X[feature].value_counts()) != 2:
                X[feature] = X[feature].map(self.m[feature])  
        X[self.bin_features_col] = X[self.bin_features_col].apply(lambda x: x.map(self.m['bin']))

        return X

In [281]:
imputer = FeatureImputer()

imputer.fit(x_train)

x_test = imputer.transform(x_test)
x_train = imputer.transform(x_train)

In [282]:
def create_bootstrap_samples(data: np.array, n_samples: int = 1000) -> np.array:
    """
    Создание бутстреп-выборок.

    Parameters
    ----------
    data: np.array
        Исходная выборка, которая будет использоваться для
        создания бутстреп выборок.

    n_samples: int, optional, default = 1000
        Количество создаваемых бутстреп выборок.
        Опциональный параметр, по умолчанию, равен 1000.

    Returns
    -------
    bootstrap_idx: np.array
        Матрица индексов, для создания бутстреп выборок.

    """
    bootstrap_idx = np.random.randint(
        low=0, high=len(data), size=(n_samples, len(data))
    )
    return bootstrap_idx


def create_bootstrap_metrics(y_true: np.array,
                             y_pred: np.array,
                             metric: callable,
                             n_samlpes: int = 1000) -> List[float]:
    """
    Вычисление бутстреп оценок.

    Parameters
    ----------
    y_true: np.array
        Вектор целевой переменной.

    y_pred: np.array
        Вектор прогнозов.

    metric: callable
        Функция для вычисления метрики.
        Функция должна принимать 2 аргумента: y_true, y_pred.

    n_samples: int, optional, default = 1000
        Количество создаваемых бутстреп выборок.
        Опциональный параметр, по умолчанию, равен 1000.

    Returns
    -------
    bootstrap_metrics: List[float]
        Список со значениями метрики качества на каждой бустреп выборке.

    """
    scores = []

    if isinstance(y_true, pd.Series):
        y_true = y_true.values

    bootstrap_idx = create_bootstrap_samples(y_true)
    for idx in bootstrap_idx:
        y_true_bootstrap = y_true[idx]
        y_pred_bootstrap = y_pred[idx]

        score = metric(y_true_bootstrap, y_pred_bootstrap)
        scores.append(score)

    return scores


def calculate_confidence_interval(scores: list, conf_interval: float = 0.95) -> Tuple[float]:
    """
    Вычисление доверительного интервала.

    Parameters
    ----------
    scores: List[float / int]
        Список с оценками изучаемой величины.

    conf_interval: float, optional, default = 0.95
        Уровень доверия для построения интервала.
        Опциональный параметр, по умолчанию, равен 0.95.

    Returns
    -------
    conf_interval: Tuple[float]
        Кортеж с границами доверительного интервала.

    """
    left_bound = np.percentile(
        scores, ((1 - conf_interval) / 2) * 100
    )
    right_bound = np.percentile(
        scores, (conf_interval + ((1 - conf_interval) / 2)) * 100
    )

    return left_bound, right_bound

In [283]:
class Adversarial_validation:
    
    def __init__(self, model):
        self.model = model
        self.y_pred = None
    
    def fit(self, X_train, X_test):
        x_adv = pd.concat([x_train, x_test], axis=0)
        y_adv = np.hstack((np.zeros(x_train.shape[0]), np.ones(x_test.shape[0])))
        assert x_adv.shape[0] == y_adv.shape[0]
        
        x_adv = x_adv.drop(['TransactionID', 'TransactionDT'], axis=1)
        
        self.model.fit(x_adv, y_adv)
        
        y_pred_adv = self.model.predict_proba(x_adv)
        score = roc_auc_score(y_adv, y_pred_adv[:, 1])
        print(round(score, 4))
        
        x_t = X_train.copy().drop(['TransactionID', 'TransactionDT'], axis=1)
        self.y_pred = self.model.predict_proba(x_t)
        
    def split(self, X, y):
        
        X_valid, y_valid = X[self.y_pred[:, 1] >= 0.5], y[self.y_pred[:, 1] >= 0.5]
        X_train, y_train = X[self.y_pred[:, 1] < 0.5], y[self.y_pred[:, 1] < 0.5]
        x_valid.shape
        
        return X_train, y_train, X_valid, y_valid

In [257]:
av = Adversarial_validation(xgb.XGBClassifier(n_estimators=10))
av.fit(x_train, x_test)

0.8863


In [284]:
def test_features(x_train, y_train):

    x_train_base, y_train_base, x_valid, y_valid = av.split(x_train, y_train)

    dtrain = xgb.DMatrix(
            data=x_train_base, label=y_train_base
        )
    dvalid = xgb.DMatrix(
            data=x_valid, label=y_valid
        )

    dtrain_all = xgb.DMatrix(
            data=x_train, label=y_train
        )
    
    params = {
        "booster": "gbtree",
        "objective": "binary:logistic",
        "eval_metric": "auc",
        "learning_rate": 0.1,
        "n_estimators": 1000,
        "reg_lambda": 100,
        "max_depth": 4,
        "gamma": 10,
        "nthread": 6,
        "seed": 27
        }

    final_model = xgb.train(
            params=params,
            dtrain=dtrain,
            num_boost_round=10000,
            early_stopping_rounds=50,
            evals=[(dtrain, "train"), (dvalid, "valid")],
            verbose_eval=None,
            maximize=True,
        )


    cv_result_cat = xgb.cv(
        params=params,
        dtrain=dtrain_all,
        num_boost_round=10000,
        early_stopping_rounds=50,
        verbose_eval=None,
        stratified=True,
        metrics="auc",
        maximize=True,
        shuffle=True,
        nfold=3,
    )
    
    train_score = roc_auc_score(y_train_base, final_model.predict(dtrain))
    test_score = roc_auc_score(y_valid, final_model.predict(dvalid))

    scores = create_bootstrap_metrics(y_valid, final_model.predict(dvalid), roc_auc_score)

    print(f"Train-score: {round(train_score, 3)}, Test-score: {round(test_score, 3)}")
    print(f"CV-results: {cv_result_cat.loc[cv_result_cat.index[-1], ['test-auc-mean', 'test-auc-std'] ]}")
    print(calculate_confidence_interval(scores))
    
    return final_model

In [222]:
final_model = test_features(x_train, y_train)

[23:55:16] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[23:57:18] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[23:57:19] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in la

Задание 1: признак TransactionDT - это смещение в секундах относительно базовой даты. Базовая дата - 2017-12-01, преобразовать признак TransactionDT в datetime, прибавив к базовой дате исходное значение признака. Из полученного признака выделить год, месяц, день недели, час, день.

In [285]:
base_name = 'TransactionDT'
base = '2017-12-01'
def datetime_transform(df, feature_name, base):
    X = df.copy()
    X[f'{feature_name}_time'] = base
    X[f'{feature_name}_time'] = pd.to_datetime(X[f'{feature_name}_time']) + pd.to_timedelta(X[feature_name], unit='s')
    X.set_index(f'{feature_name}_time', inplace=True)
    X['Year'] = X.index.year
    X['Month'] = X.index.month
    X['Day of week'] = X.index.dayofweek
    X['Day'] = X.index.day
    X['Hour'] = X.index.hour
    X.reset_index(inplace=True)
    X.drop(f'{feature_name}_time', axis=1, inplace=True)
    return X
x_train = datetime_transform(x_train, base_name, base)
x_train.head(2)

,TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,...,V335,V336,V337,V338,V339,Year,Month,Day of week,Day,Hour
0,2987000,86400,68.5,4,13926,NaN,150.0,1.0,142.0,1.0,...,NaN,NaN,NaN,NaN,NaN,2017,12,5,2,0
1,2987001,86401,29.0,4,2755,404.0,150.0,2.0,102.0,1.0,...,NaN,NaN,NaN,NaN,NaN,2017,12,5,2,0


In [259]:
final_model = test_features(x_train, y_train)

[00:15:28] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[00:17:30] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[00:17:31] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in la

Задание 2: сделать конкатенацию признаков

card1 + card2;

card1 + card2 + card_3 + card_5;

card1 + card2 + card_3 + card_5 + addr1 + addr2

Рассматривать их как категориальных признаки.

In [286]:
def new_category(df):
    X = df.copy()
    X['card1 + card2'] = X['card1'].astype('str') + ' | ' + X['card2'].astype('str') 
    X['card1 + card2 + card3 + card5'] = X['card1'].astype('str')  + ' | ' + X['card2'].astype('str')+ ' | '  + X['card3'].astype('str')  + ' | ' + X['card5'].astype('str') 
    X['card1 + card2 + card3 + card5 + addr1 + addr2'] = X['card1'].astype('str')  + ' | ' + X['card2'].astype('str')  + ' | '+ X['card3'].astype('str')  + ' | ' + X['card5'].astype('str')  + ' | ' + X['addr1'].astype('str')  + ' | ' + X['addr2'].astype('str') 
    return X
x_train = new_category(x_train)

In [306]:
cat_features = ['card1 + card2',
            'card1 + card2 + card3 + card5',
            'card1 + card2 + card3 + card5 + addr1 + addr2']

In [287]:
encoder = FrequencyEncoder(features)

encoder.fit(x_train)

x_train = encoder.transform(x_train)

In [271]:
final_model = test_features(x_train.drop(cat_features, axis=1), y_train)

[00:35:32] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[00:37:49] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[00:37:50] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in la

Задание 3: Сделать FrequencyEncoder для признаков card1 - card6, addr1, addr2.

In [272]:
class FrequencyEncoder:
    """FrequencyEncoder"""
    
    def __init__(self, categorical_features):
        self.categorical_features= categorical_features
        self.m = None
        
    def fit(self, X):
        self.m = {f: {} for f in self.categorical_features}
        for feature in self.categorical_features:
            self.m[feature] = X[feature].value_counts(normalize=True).to_dict()
    
    def transform(self, X): 
        for feature in self.categorical_features:
            X[f'{feature}_freq_encoding'] = X[feature].map(self.m[feature])

        return X

In [288]:
features = ['card1', 'card2', 'card3', 'card4',
            'card5', 'card6', 'addr1', 'addr2']
encoder = FrequencyEncoder(features)

encoder.fit(x_train)

x_train = encoder.transform(x_train)

In [291]:
final_model = test_features(x_train.drop(cat_features, axis=1), y_train)

[00:54:24] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[00:56:44] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[00:56:45] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in la

Задание 4: Создать признаки на основе отношения: 

TransactionAmt к вычисленной статистике. 

Статистика - среднее значение / стандартное отклонение TransactionAmt, 

сгруппированное по card1 - card6, addr1, addr2, и по признакам, созданным в задании 2.

In [292]:

group_list = ['card1', 'card2', 'card3', 'card4',
              'card5', 'card6', 'addr1', 'addr2',
              'card1 + card2', 'card1 + card2 + card3 + card5', 
              'card1 + card2 + card3 + card5 + addr1 + addr2']
stat_list = ['mean', 'std']
def feature_ratio(X, features, group_list, stat_list):
    agg_dict = {feature: stat_list for feature in features}
    group_dict = {}
    for group in group_list:
        df = X.groupby(group, as_index=False).agg(agg_dict)
        df.columns = [f'{group}_{"_".join(x)}' for x in df.columns.ravel()]
        df = df.rename(columns={df.columns[0]:group})
        X = X.merge(df, on=[group], how='left')
    for feature in features:
        columns = [col for col in X.columns if feature in col]
        X[columns[1:]] = np.repeat([X[columns[0]].values], repeats=len(columns)-1, axis=0).T/ X[columns[1:]].values
    return X

features = ['TransactionAmt']

x_train = feature_ratio(x_train, features, group_list, stat_list)
    

In [295]:
final_model = test_features(x_train.drop(cat_features, axis=1), y_train)

[01:03:56] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[01:06:42] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[01:06:43] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in la

Задание 5: Создать признаки на основе отношения: D15 к вычисленной статистике. Статистика - среднее значение / стандартное отклонение D15, сгруппированное по card1 - card6, addr1, addr2, и по признакам, созданным в задании 2.

In [296]:
features = ['D15']

x_train = feature_ratio(x_train, features, group_list, stat_list)

In [297]:
final_model = test_features(x_train.drop(cat_features, axis=1), y_train)

[01:13:53] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[01:17:03] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[01:17:04] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in la

Задание 6: выделить дробную часть и целую часть признака TransactionAmt в два отдельных признака. После создать отдельных признак - логарифм от TransactionAmt

In [298]:
x_train['TransactionAmt_int'] = x_train['TransactionAmt']//1
x_train['TransactionAmt_frac'] = x_train['TransactionAmt']%1
x_train['TransactionAmt_log'] = np.log(x_train['TransactionAmt'])
x_train[['TransactionAmt_int', 'TransactionAmt_frac', 'TransactionAmt_log']].head(2)

,TransactionAmt_int,TransactionAmt_frac,TransactionAmt_log
0,68.0,0.5,4.226834
1,29.0,0.0,3.367296


In [299]:
final_model = test_features(x_train.drop(cat_features, axis=1), y_train)

[01:24:26] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[01:27:41] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[01:27:42] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in la

Задание 7 (опция): выполнить предварительную подготовку / очистку признаков P_emaildomain и R_emaildomain (что и как делать - остается на ваше усмотрение) и сделать Frequency Encoding для очищенных признаков.

In [300]:
def emaildomain(X):
    X['P_emaildomain_name'] = X['P_emaildomain'].astype('str').str.split('.').apply(lambda s:s[0])
    X['P_emaildomain_domain'] = X['P_emaildomain'].astype('str').str.split('.').apply(lambda s:'.'.join(s[1:]))
    X['R_emaildomain_name'] = X['R_emaildomain'].astype('str').str.split('.').apply(lambda s:s[0])
    X['R_emaildomain_domain'] = X['R_emaildomain'].astype('str').str.split('.').apply(lambda s:'.'.join(s[1:]))
    X.loc[X['R_emaildomain_name'] == 'nan', 'R_emaildomain_name'] = np.NaN
    X.loc[X['R_emaildomain_domain'] == '', 'R_emaildomain_domain'] = np.NaN
    X.loc[X['P_emaildomain_name'] == 'nan', 'P_emaildomain_name'] = np.NaN
    X.loc[X['P_emaildomain_domain'] == '', 'P_emaildomain_domain'] = np.NaN
    X['P_R_domain'] = X['R_emaildomain_domain'] + ' | ' + X['P_emaildomain_domain']
    return X

In [301]:
x_train = emaildomain(x_train)

In [302]:
features = ['P_emaildomain_name', 'P_emaildomain_domain', 'P_R_domain',
            'R_emaildomain_name', 'R_emaildomain_domain']
encoder = FrequencyEncoder(features)

encoder.fit(x_train)

x_train = encoder.transform(x_train)

In [307]:
cat_features.extend(features)

In [309]:
final_model = test_features(x_train.drop(cat_features, axis=1), y_train)

[01:38:44] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[01:41:42] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[01:41:43] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in la

### Выводы ###

0 __Базовое состояние датасета (с прошлого домашнего задания) __

Train-score: 0.92, Test-score: __0.86__

CV-results: test-auc-mean    __0.903114__
            test-auc-std     0.001776

(0.8142575724063467, 0.8999876611157724)

1 __Score после преобразования заты и выделения из неё дней, часов и т.д.__

Ничего не изменилось, возможно данные признаки сами по себе не несут много информации. Их можно попробовать использовать для признаков основанных на группировках.

Train-score: 0.92, Test-score: 0.859

CV-results: test-auc-mean    0.903629
            test-auc-std     0.001906

(0.8140769337182461, 0.899805274012838)

2 __Score после создания признаков на основе канкотинации категориальных признаков (эти признаки закодированы частотным кодированием) __

Скор стал хуже

Train-score: 0.918, Test-score: 0.859

CV-results: test-auc-mean    0.899795
            test-auc-std     0.002553

(0.8148672327407076, 0.8989859650155663)

3 __Score после частотного кодирования признаков card.. и addr.. __

По сравнениию с базовой моделью score вырос на 0,5%, качество на кроссвалидации выросло на 0,2%

Train-score: 0.922, Test-score: 0.866

CV-results: test-auc-mean    0.905859
            test-auc-std     0.001455

(0.8210098183584115, 0.9036691552955908)

4 __Score после создания признаков на основе отношения: TransactionAmt к вычисленной статистике__

По сравнениию с п.3 качество на кроссвалидации выросло ещё на 0,2%, но качество на отложенной выборке опять упало и доверительный интервал сместиля в меньшую сторону

Train-score: 0.924, Test-score: 0.86

CV-results: test-auc-mean    0.907595
            test-auc-std     0.002286

(0.8155875085185057, 0.8994701391908796)

5 __Score после создания признаков на основе отношения: D15 к вычисленной статистике__

По сравнениию с п.4 качество на кроссвалидации почти не изменилось, а качество на отложенной выборке увеличелось, так же как и границы доверительного интервала, хотя в п.3 качество всё ещё лучше

Train-score: 0.924, Test-score: 0.862

CV-results: test-auc-mean    0.907318
            test-auc-std     0.002042

(0.8183582731803614, 0.9015484872240581)

6 __Score после преобразований TransactionAmt__

Стало незначительно хуже, по сравнению с п 5.

Train-score: 0.926, Test-score: 0.862

CV-results: test-auc-mean    0.906850
            test-auc-std     0.001802

(0.8179520422450302, 0.900944992084082)

7 __Score после обработки emaildomain__

Проверила гипотизу о том что страна емейла и сайт емейла имеют значение. Частотная кодировка качество только ухудшила. Но эти признаки можно закодировать иначе, например сруппировать по ним статистики или вычеслить среднее по таргету для каждого из признаков (естественно убрав те, которые встречаются меньше 15 раз или около того, чтобы избежать переобучения)

Train-score: 0.924, Test-score: 0.858

CV-results: test-auc-mean    0.907309
            test-auc-std     0.002236

(0.8140861747438145, 0.8976924670714873)